In [ ]:
# third party
import matplotlib.pyplot as plt

# relative import
from mnist_dataset import mnist
from mnist_dataset import mnist_raw
import numpy as np

# syft absolute
import syft as sy

print(f"{sy.__version__ = }")

## 1. Launch the domain, upload the data

In [9]:
# node = sy.orchestra.launch(name="mnist-domain", dev_mode=True, reset=True)
root_client = sy.login(email="info@openmined.org", password="changethis",port=8081)

Logged into <test_domain: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

### Load the MNIST dataset

Let's load the raw MNIST images and show with the `mnist_raw` function from [`mnist_datasets.py`](./datasets.py)

In [ ]:
train_images, train_labels, _, _ = mnist_raw()

In [ ]:
print("train image num: "+str(len(train_images)))
plt.imshow(train_images[0])

In [ ]:
train_labels[0]

In [ ]:
print(f"{train_images.shape = }")
print(f"{train_labels.shape = }")

### Processing: Flattening the MNIST images and apply one-hot encoding on the labels

In [ ]:
train_images, train_labels, _, _ = mnist()

### Get a subset of MNIST

In [ ]:
num_samples = 1000

In [ ]:
train_images = train_images[:num_samples, :]
train_labels = train_labels[:num_samples, :]

In [ ]:
print(f"{train_images.shape = }")
print(f"{train_labels.shape = }")

The `train_images` and `train_labels` are the private data. Let's create similar mock data with the same shape

In [ ]:
mock_images = np.random.rand(num_samples, 784)
mock_images.shape

In [ ]:
mock_labels = np.eye(10)[np.random.choice(10, num_samples)]
mock_labels.shape

In [ ]:
assert mock_labels.shape == train_labels.shape
assert mock_images.shape == train_images.shape

### The DO uploads the data

In [ ]:
dataset = sy.Dataset(
    name="MNIST data",
    description="""Contains the flattened training images and one-hot encoded training labels.""",
    url="https://storage.googleapis.com/cvdf-datasets/mnist/",
)

dataset.add_contributor(
    role=sy.roles.UPLOADER,
    name="Alice",
    email="alice@openmined.com",
    note="Alice is the data engineer at the OpenMined",
)

dataset.contributors

In [ ]:
asset_mnist_train_input = sy.Asset(
    name="MNIST training images",
    description="""The training images of the MNIST dataset""",
    data=train_images,
    mock=mock_images,
)

asset_mnist_train_labels = sy.Asset(
    name="MNIST training labels",
    description="""The training labels of MNIST dataset""",
    data=train_labels,
    mock=mock_labels,
)

dataset.add_asset(asset_mnist_train_input)
dataset.add_asset(asset_mnist_train_labels)

In [ ]:
root_client.upload_dataset(dataset)

### The DO inspects the uploaded data

In [10]:
datasets = root_client.api.services.dataset.get_all()
# assert len(datasets) == 1
datasets

SyftWarning: You're performing an operation on high side domain, which could host datasets with private information.

DictTuple()

#### The first asset of the dataset contains the training and mock images

In [11]:
datasets[0].assets[0]

IndexError: tuple index out of range

#### The second asset contains the training and mock labels

In [12]:
datasets[0].assets[1]

IndexError: tuple index out of range

### The DO creates an account for the Data Scientist (DS)

In [ ]:
register_result = root_client.register(
    name="Sheldon Cooper",
    email="sheldon@caltech.edu",
    password="changethis",
    password_verify="changethis",
    institution="Caltech",
    website="https://www.caltech.edu/",
)

In [ ]:
assert isinstance(register_result, sy.SyftSuccess)

### 📓 Now switch to the [first DS's notebook](./01-data-scientist-submit-code.ipynb)